In [ ]:
import numpy as np

# needleman_wunsch width
match = 1
mismatch = 1
gap = 2

class WordProxy:
    __simplestopwords = set(',.?!:')
    __EMPTY = None
    
    def __init__(self, word):
        self.wp = word.strip()
        if len(self.wp) > 1 and self.wp[-1] in WordProxy.__simplestopwords:
            self.w = self.wp[:-1]
            self.p = self.wp[-1]
        else :
            self.w = self.wp
            self.p = None
            
    def __str__(self):
        return self.wp
    
    def __repr__(self):
        return self.wp
    
    def empty():
        if WordProxy.__EMPTY is None:
            WordProxy.__EMPTY = WordProxy('')
        return WordProxy.__EMPTY
        
        
    def getWP(self):
        return self.wp
    
    def getW(self):
        return self.w
    
    def hasP(self):
        return not self.p is None
        
    def getP(self):
        return self.p
    
    
    def iso(self, other):
        return self.w.lower() == other.w.lower() 

        
    def isEmpty(self):
        return self ==  WordProxy.empty()
        
#Methods Diffs Algo -------------------------------------------------------------------------------
def score(x, y):
    if x.iso(y):
        return match
    else:
        return -mismatch
     
    
def needleman_wunsch(image, shadow):
    m=len(image)
    n=len(shadow)
    F = np.zeros((m  , n ))
    F[:,0] = -np.arange(0, m*gap,gap)
    F[0,:] = -np.arange(0, n*gap,gap)
            
    for i in range(1,m):
        for j in range(1,n):
            s = score(image[i], shadow[j])
            F[i][j] = max(F[i-1][j-1] + s, F[i-1][j] - gap, F[i][j-1] - gap)
    return F



def nw_traceback(image , shadow, F):
    m=len(F)
    n=len(F[0])
    outImage=[]
    outShadow=[]
    i,j=m-1,n-1
    while i>=0 and j>=0:
        matching = image[i].iso(shadow[j])
        if F[i-1][j]==F[i][j-1] or matching:
            if matching:
                outImage.append(image[i])
                outShadow.append(shadow[j])
            else :
                outImage.append(image[i])
                outImage.append(WordProxy.empty())
                outShadow.append(WordProxy.empty())
                outShadow.append(shadow[j])
            i-=1
            j-=1
            
        elif F[i-1][j]>F[i][j-1]:
            outImage.append(image[i])
            outShadow.append(WordProxy.empty())
            i-=1
        else:
            outImage.append(WordProxy.empty())
            outShadow.append(shadow[j])
            j-=1
    if i>0:
        while i>=0:
            if image[i].iso(shadow[j]):
                outImage.append(image[i])
                outShadow.append(shadow[j])
                i-=1
                j-=1
            else:
                outImage.append(image[i])
                outShadow.append(WordProxy.empty())
                i-=1
    if j>0:
        while j>=0:
            if shadow[j]==image[i]:
                outImage.append(image[i])
                outShadow.append(shadow[j])
                i-=1
                j-=1
            else:
                outImage.append(WordProxy.empty())
                outShadow.append(shadow[j])
                j-=1
    if i==0 and j==0:
        outImage.append(image[i])
        outShadow.append(shadow[j])
    elif i==0:
        outImage.append(image[i])
        outShadow.append(WordProxy.empty())
    elif j==0:
        outImage.append(WordProxy.empty())
        outShadow.append(shadow[j])

    return outImage,outShadow


    
def processSequences(wim, wsh):                  
    F = needleman_wunsch(wim,wsh)
    outImage,outShadow = nw_traceback(wim, wsh, F)
    outImage = outImage[::-1]
    outShadow = outShadow[::-1]

    binMatch = []
    for i in range(len(outImage)):
        if not (outImage[i].isEmpty() or  outShadow[i].isEmpty()):
            binMatch.append(1)
        else:
            binMatch.append(0)
    cbranch = []
    ibranch = []
    sbranch = []
    res = []
    image = []
    shadow = []
    bl = len(binMatch)-1
    
    
    for i in range(bl+1):
        if binMatch[i] == 0 :
            if i == 0 or (i>0 and binMatch[i-1] == 1):
                image.append('<span class="im">') 
                shadow.append('<span class="sh">') 
            if not outImage[i].isEmpty():
                image.append(outImage[i].getWP())
            if not outShadow[i].isEmpty():
                shadow.append(outShadow[i].getWP())
            if i == bl or (i < bl and binMatch[i+1] == 1):   
                ibranch.extend(image[1:])
                sbranch.extend(shadow[1:])
                image.append('</span>') 
                shadow.append('</span>') 
                res.extend(shadow)
                res.extend(image)
                image = []
                shadow = []
        else:
            cbranch.append(outImage[i].getWP())
            if outImage[i].getP() == outShadow[i].getP():
                res.append(outImage[i].getWP())
            else:
                diffP = outImage[i].getW()
                if outShadow[i].hasP() :
                    diffP += '<span class="sh">'+outShadow[i].getP()+ '</span>'
                if outImage[i].hasP() :
                    diffP += '<span class="im">'+outImage[i].getP()+ '</span>'
                res.append(diffP)
    
    pRes = ' '.join(res).split('<br>')
    pRes = list(map(lambda p: "<p>"+p+"</p>", pRes))
    strRes = ' '.join(pRes)
    
    strCom = ' '.join(cbranch)
    strNew = ' '.join(ibranch) 
    strOld = ' '.join(sbranch) 
   
    return strRes, strCom, strNew, strOld

     
# p0 = 'Hello Word i m Hassan, its boolcheat'
# p1 = 'Hello Word you are Hassan? its boolcheat'

# wsh = list(map(lambda x : WordProxy(x), p0.split(' ')))
# wim = list(map(lambda x : WordProxy(x), p1.split(' ')))
           
# res, fix, new, old = processSequences(wim, wsh)          
# print(res)        


import nltk
nltk.download()